<a href="https://colab.research.google.com/github/vasukokkiligadda/plantpathology/blob/master/tomato_diseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>